# Chains in LangChain
- Chains are invaluable due to their capacity to effortlessly blend diverse components, shaping a singular and coherent application. 
- Through the creation of chains, multiple elements can seamlessly come together.
-  A chain is crafted to take in user input, polish it using a PromptTemplate, and subsequently pass on this refined response to a large language model (LLM).

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

## Importing Libraries and Basic worflow setup

In [ ]:
#!pip install pandas
#!pip install --upgrade langchain
#!pip install openai

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain
from langchain.chains import SequentialChain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

Note: LLM's do not always produce the same results.

In [1]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [3]:
import pandas as pd
df = pd.read_csv('Data.csv')

In [4]:
df.head()

,Product,Review
0,GlamourGlow Lipstick,GlamourGlow lipstick is fantastic! The color i...
1,RubyRose Lipstick,I was disappointed with RubyRose lipstick. The...
2,SilkShimmer Lipstick,SilkShimmer lipstick exceeded my expectations!...
3,CrimsonCharm Lipstick,CrimsonCharm lipstick is a miss for me. The co...
4,VelvetVogue Lipstick,VelvetVogue lipstick is amazing! It glides on ...


## LLMChain
- The LLMChain represents the most basic form of chain within this system.
- Within this setup, the LLMChain accepts various input parameters.
    - It employs the PromptTemplate to transform these inputs into a coherent prompt. 
    - This polished prompt is then inputted into the model. 
    - After receiving the output, the LLMChain uses the OutputParser, if provided, to further refine and format the result into its ultimate usable form.
- 

In [ ]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "How can you describe this {product} in a single word?"
)

In [ ]:
chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
product = "CrimsonCharm Lipstick"
chain.run(product)

*Output would look like this:<br>*

**> Entering new LLMChain chain...<br>**
Prompt after formatting:<br>
<span style="color:green">Human: How can you describe this CrimsonCharm Lipstick in a single word?</span>

**> Finished chain.<br>**

'Bold.'


## SequentialChain
- A sequential chain is a chain that combines various individual chains, where the output of one chain serves as the input for the next in a continuous sequence. 
- It operates by running a series of chains consecutively.

## SimpleSequentialChain
- Sequential chains, in their simplest form, consist of steps where each step takes one input and produces one output. 
- The output from one step becomes the input for the next.
- This sequential approach ensures systematic and efficient handling of data, making it ideal for scenarios where a linear flow of information processing is essential.

In [ ]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [ ]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [ ]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [ ]:
overall_simple_chain.run(product)

*Output would look like this:*

**> Entering new SimpleSequentialChain chain...</br>**
<span style="color:crimson">CrimsonCharm Cosmetics.</span></br>
<span style="color:blue">CrimsonCharm Cosmetics is a luxury makeup brand that offers a range of high-quality beauty products, including lipstick, eyeshadow, and foundation. The brand is known for its rich, bold colors and impeccable formulations, which are designed to enhance and beautify any complexion. With a commitment to using only the finest ingredients and state-of-the-art manufacturing techniques, CrimsonCharm Cosmetics has gained a loyal following among beauty enthusiasts and professionals alike. Whether you're looking for a bold, dramatic look or something more subtle and sophisticated, you're sure to find the perfect product at CrimsonCharm Cosmetics. Discover the magic of this brand, and experience the ultimate in beauty and luxury.</span>

**> Finished chain.</br>**
'CrimsonCharm Cosmetics is a luxury makeup brand that offers a range of high-quality beauty products, including lipstick, eyeshadow, and foundation. The brand is known for its rich, bold colors and impeccable formulations, which are designed to enhance and beautify any complexion. With a commitment to using only the finest ingredients and state-of-the-art manufacturing techniques, CrimsonCharm Cosmetics has gained a loyal following among beauty enthusiasts and professionals alike. Whether you're looking for a bold, dramatic look or something more subtle and sophisticated, you're sure to find the perfect product at CrimsonCharm Cosmetics. Discover the magic of this brand, and experience the ultimate in beauty and luxury.'

## SequentialChain
- Not all of the sequential chains operate with a single string input and output. In more intricate setups, these chains handle multiple inputs and generate multiple final outputs. 
- The careful naming of input and output variables holds important significance in these complex chains.
- A more general form of sequential chains allows for multiple inputs/outputs. Any step in the chain can take in multiple inputs.

In [ ]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )


In [ ]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [ ]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [ ]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [ ]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [ ]:
review = df.Review[5]
overall_chain(review)

*Output would look like this:*

**> Entering new SequentialChain chain...**


**> Entering new LLMChain chain...**</br>
Prompt after formatting:</br>
<span style="color:green">Human: Translate the following review to english:</span>

<span style="color:green">PetalPink lipstick looked lovely in the tube, but the color payoff was disappointing. It took several layers to get the desired hue, and it didn't last long.</span>

**> Finished chain.**


**> Entering new LLMChain chain...**</br>
Prompt after formatting:</br>
<span style="color:green">Human: Can you summarize the following review in 1 sentence:</span>

<span style="color:green">El lápiz labial PetalPink se veía encantador en el tubo, pero el color no fue satisfactorio. Se necesitaron varias capas para obtener el tono deseado y no duró mucho.</span>

<span style="color:green">The PetalPink lipstick looked lovely in the tube, but the color payoff was disappointing. It took several layers to get the desired hue, and it didn't last long.</span>

**> Finished chain.**


**> Entering new LLMChain chain...**</br>
Prompt after formatting:</br>
<span style="color:green">Human: What language is the following review:</span>

<span style="color:green">PetalPink lipstick looked lovely in the tube, but the color payoff was disappointing. It took several layers to get the desired hue, and it didn't last long.</span>

**> Finished chain.**


**> Entering new LLMChain chain...**</br>
Prompt after formatting:</br>
<span style="color:green">Human: Write a follow up response to the following summary in the specified language:</span>

<span style="color:green">Summary: The PetalPink lipstick was disappointing due to its poor color payoff and short wear time.</span>

<span style="color:green">Language: English</span>

**> Finished chain.**

**> Finished chain.**</br>

{'Review': "PetalPink lipstick looked lovely in the tube, but the color payoff was disappointing. It took several layers to get the desired hue, and it didn't last long.",</br>
 'English_Review': "El lápiz labial PetalPink se veía encantador en el tubo, pero el color no fue satisfactorio. Se necesitaron varias capas para obtener el tono deseado y no duró mucho. \n\nThe PetalPink lipstick looked lovely in the tube, but the color payoff was disappointing. It took several layers to get the desired hue, and it didn't last long.",</br>
 'summary': 'The PetalPink lipstick was disappointing due to its poor color payoff and short wear time.',</br>
 'followup_message': "Thank you for your feedback on the PetalPink lipstick. We apologize that you were disappointed with the color payoff and wear time of our product. We value our customers' opinions and will take your feedback into consideration as we continue to improve our offerings. Please feel free to reach out to us with any further comments or concerns."}

## Router Chain

- The Router Chain is used for complicated tasks. If we have multiple subchains, each of which is specialized for a particular type of input, we could have a router chain that decides which subchain to pass the input to.
- It consists of:
    - **Router Chain:** It is responsible for selecting the next chain to call.
    - **Destination Chains:** Chains that the router chain can route to.
    - **Default chain:** Used when the router can’t decide which subchain to use.
- This involves directing an input toward a specific chain based on what exactly that input is. 
- When there are several subchains, each tailored for distinct input types, a router chain comes into play. 
- This router chain acts as a decision-maker, determining which specialized subchain to send the input to. 
- Essentially, it enables the seamless routing of inputs to the appropriate subchains, ensuring efficient and precise processing based on the input’s specific characteristics.

In [ ]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [ ]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [ ]:
llm = ChatOpenAI(temperature=0, model=llm_model)

In [ ]:

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [ ]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [ ]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [ ]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [ ]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [ ]:
chain.run("What is black body radiation?")

*Output would like this:*

**> Entering new MultiPromptChain chain...**</br>
physics: {'input': 'What is black body radiation?'}

**> Finished chain.**</br>
"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an object that absorbs all radiation that falls on it and emits radiation at all wavelengths. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This type of radiation is important in understanding the behavior of stars, as well as in the development of technologies such as incandescent light bulbs and infrared cameras."

In [ ]:
chain.run("what is 2 + 2")

**> Entering new MultiPromptChain chain...**</br>
math: {'input': 'what is 2 + 2'}

**> Finished chain.**</br>
'As an AI language model, I can answer this question easily. The answer to 2 + 2 is 4.'